In [50]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [51]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [52]:
# Open Sheet
sheet = client.open_by_key("1Feh5lJStiEkMjT0qkLl8yu81Tbi1WjIeILcxrk13FsI") # Master Sheets ( Dimensions Sheet)

#### Extracting Masters from GoogleSheetBackend Masters

In [53]:
StoreMaster = sheet.worksheet("StoreMaster")
SOSTargetMaster = sheet.worksheet("SOSTargetMaster")
ProductMaster = sheet.worksheet("ProductMaster")
Overall_Master_Product = sheet.worksheet("Overall_Master_Product")

NPF_Master_SOS = sheet.worksheet("NPF_Master_SOS")


In [54]:
StoreMaster = get_as_dataframe(StoreMaster)
SOSTargetMaster = get_as_dataframe(SOSTargetMaster)
ProductMaster = get_as_dataframe(ProductMaster)

Overall_Master_Product = get_as_dataframe(Overall_Master_Product)
NPF_Master_SOS = get_as_dataframe(NPF_Master_SOS)


In [55]:
StoreMasterRevised = StoreMaster[
    ['LIS Store Code', 'Old DT Store Code', 'DT Store Code', 'Amazing ID', 'Emp User ID','LIS User Id','Employee Name']
]

StoreMasterRevised = StoreMasterRevised.rename(
    columns={
        'LIS Store Code': 'New_DT_Store_Code',
        'Old DT Store Code': 'OLD_DT_Store_Code',
        'DT Store Code': 'LIS_StoreCode',
        'Amazing ID': 'Old_Amazing_EMP_ID',
        'Emp User ID': 'New_Amazing_EMP_ID',
        'LIS User Id':'LISUserId',
        'Employee Name':'EmployeeName'
    }
)

In [56]:
## NEED TO HAVE CAMP ID ALWYAS IN UPPER CASE IN MASTER ANN COVERAGE AND SO ON

StoreMasterRevised['New_Amazing_EMP_ID'] = (
    StoreMasterRevised['New_Amazing_EMP_ID']
    .astype(str)
    .str.upper()
)

StoreMasterRevised['Old_Amazing_EMP_ID'] = (
    StoreMasterRevised['Old_Amazing_EMP_ID']
    .astype(str)
    .str.upper()
)

In [57]:
StoreMasterRevised.head()

New_DT_Store_Code OLD_DT_Store_Code  LIS_StoreCode Old_Amazing_EMP_ID  \
0       DT02DMA2001       DT02DMA2001  CLG-Store-497               X157   
1       DT02DMA2003       DT02DMA2003  CLG-Store-491          IN0028M57   
2       DT02DMA2005       DT02DMA2005  CLG-Store-494         IN0028M154   
3         DT02MET54       DT02MET54-1  CLG-Store-499         IN0028M475   
4       DT02RRLFRDF       DT02RRLFRDF  CLG-Store-496         IN0028M326   

  New_Amazing_EMP_ID    LISUserId    EmployeeName  
0            CAMP059  CLG-ISP-507    Mukesh Kumar  
1            CAMP061  CLG-ISP-926  Shankar Mandal  
2            CAMP062  CLG-ISP-504    Sachin Kumar  
3            CAMP112  CLG-ISP-509      Firoz Alam  
4            CAMP132  CLG-ISP-498           Vimal

In [58]:
StoreMasterRevised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   New_DT_Store_Code   837 non-null    object
 1   OLD_DT_Store_Code   808 non-null    object
 2   LIS_StoreCode       837 non-null    object
 3   Old_Amazing_EMP_ID  837 non-null    object
 4   New_Amazing_EMP_ID  837 non-null    object
 5   LISUserId           837 non-null    object
 6   EmployeeName        837 non-null    object
dtypes: object(7)
memory usage: 45.9+ KB


#### SOS

In [59]:
sos=pd.read_csv('SOS.csv', usecols = ['Route Date','Store Code','Store Name','NPG Code','NPG Description','Face On','End on','Stand On'])
# sos=pd.read_csv('SOS.csv')

In [60]:
sos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150342 entries, 0 to 150341
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Store Code       150342 non-null  object 
 1   Store Name       150342 non-null  object 
 2   Route Date       150342 non-null  object 
 3   NPG Code         150342 non-null  object 
 4   NPG Description  150341 non-null  object 
 5   Face On          150341 non-null  float64
 6   End on           150341 non-null  float64
 7   Stand On         150341 non-null  float64
dtypes: float64(3), object(5)
memory usage: 9.2+ MB


In [61]:
sos.head()

Store Code                                     Store Name Route Date  \
0   PC000931433                     Rss_kl_elite_super_trissur  02-Jan-26   
1   DT04TESSM52                           Tescotes Immamdahali  02-Jan-26   
2    DT04RNPA43                            Ratnadeepfilm Nagar  02-Jan-26   
3    C000029806                              J S  Super Market  02-Jan-26   
4  DT04APO15152  Apollo Pharmaapo-green Glen Layout; Bellandur  02-Jan-26   

        NPG Code                  NPG Description  Face On  End on  Stand On  
0  CP1213COZZA03       TB Col ZigZag Base M/S PO3      1.0     0.0       0.0  
1  PA1112PJDKA05  TP Patjli DanKan Base 200g+100g      1.0     0.0       0.0  
2     COMP100005                         TP Dabur      3.0     0.0       0.0  
3  CP1311COPXA05         MW Col Plax PepMin 100ml      1.0     0.0       0.0  
4  CP1123COVWA02    TP Col VisWht Base Daily 100g      0.0     0.0       1.0

In [62]:
sos_npf = pd.merge(sos,NPF_Master_SOS,left_on = 'NPG Code', right_on = 'NPG Code', how = 'left')

In [63]:
sos_npf['Brand2'] = np.where(sos_npf['Brand'] == 'Colgate', 'Colgate', 'Others')

In [64]:
sos_npf_master = pd.merge(sos_npf,StoreMasterRevised,left_on = 'Store Code', right_on = 'New_DT_Store_Code', how = 'inner')

In [65]:
columns_retained = ['Route Date','New_Amazing_EMP_ID','LISUserId','EmployeeName','Store Code','Store Name','Category','Brand2','Face On','End on','Stand On']

In [66]:
sos_npf_master = sos_npf_master[columns_retained]

In [72]:
sos_npf_master['facing'] = sos_npf_master.apply(
    lambda r: (r['Face On'] or 0)*1 + (r['End on'] or 0)*0.33 + (r['Stand On'] or 0)*0.33,
    axis=1
)

In [75]:
sos_npf_master= sos_npf_master.drop(columns = ['Face On','End on','Stand On'])

In [76]:
sos_npf_master.head()

Route Date New_Amazing_EMP_ID     LISUserId             EmployeeName  \
0  02-Jan-26            CAMP464    CLG-ISP-53                     Vinu   
1  02-Jan-26            CAMP710  CLG-ISP-1238  Pichholla Ramanjaneyulu   
2  02-Jan-26            CAMP611   CLG-ISP-196      Varsha Prabhu Yanni   
3  02-Jan-26            CAMP744    CLG-ISP-24             Anjali Dutta   
4  02-Jan-26            CAMP174   CLG-ISP-488     Uddesh Singh Chouhan   

      Store Code                  Store Name    Category   Brand2  facing  
0    PC000931433  Rss_kl_elite_super_trissur  Toothbrush  Colgate     1.0  
1  DT04DMA5010_1     Hydrabad - Sanath Nagar  Toothpaste   Others     5.0  
2    DT01DMA4182  4182 Khed Pune Maharashtra    PCP / HC   Others     0.0  
3    DT03RRLFRBV  Hiland Park-the Metropolis  Toothbrush   Others     1.0  
4    DT02RRLTP76             Saraswati Nagar  Toothpaste   Others    11.0

In [78]:
summary_sos = (
    sos_npf_master
    .pivot_table(
        index=[
            'Route Date',
            'New_Amazing_EMP_ID',
            'LISUserId',
            'EmployeeName',
            'Store Code',
            'Store Name',
            'Category'
        ],
        columns='Brand2',
        values='facing',
        aggfunc='sum',
        fill_value=0
    )
    .reset_index()
)
# summary_df.columns.name = None

In [80]:
summary_sos['Total_Facings'] = summary_sos['Colgate']+summary_sos['Others']

In [81]:
summary_sos.head()

Brand2 Route Date New_Amazing_EMP_ID     LISUserId  EmployeeName   Store Code  \
0       02-Jan-26            CAMP002  CLG-ISP-1340  DIPESH YADAV  DT03DMA4903   
1       02-Jan-26            CAMP002  CLG-ISP-1340  DIPESH YADAV  DT03DMA4903   
2       02-Jan-26            CAMP002  CLG-ISP-1340  DIPESH YADAV  DT03DMA4903   
3       02-Jan-26            CAMP002  CLG-ISP-1340  DIPESH YADAV  DT03DMA4903   
4       02-Jan-26            CAMP006    CLG-ISP-05   Jyoti Kujur  DT03RRL2302   

Brand2               Store Name    Category  Colgate  Others  Total_Facings  
0       Ring Road Raipur D-mart   Mouthwash      3.0     3.0            6.0  
1       Ring Road Raipur D-mart    PCP / HC      0.0    16.0           16.0  
2       Ring Road Raipur D-mart  Toothbrush     20.0     6.0           26.0  
3       Ring Road Raipur D-mart  Toothpaste     42.0    17.0           59.0  
4            Seagull Appartment   Mouthwash     12.0     3.0           15.0

In [88]:
summary_sos.drop(columns=['Others'], inplace=True)
summary_sos.head()

Brand2 Route Date New_Amazing_EMP_ID     LISUserId  EmployeeName   Store Code  \
0       02-Jan-26            CAMP002  CLG-ISP-1340  DIPESH YADAV  DT03DMA4903   
1       02-Jan-26            CAMP002  CLG-ISP-1340  DIPESH YADAV  DT03DMA4903   
2       02-Jan-26            CAMP002  CLG-ISP-1340  DIPESH YADAV  DT03DMA4903   
3       02-Jan-26            CAMP002  CLG-ISP-1340  DIPESH YADAV  DT03DMA4903   
4       02-Jan-26            CAMP006    CLG-ISP-05   Jyoti Kujur  DT03RRL2302   

Brand2               Store Name    Category  Colgate  Total_Facings  
0       Ring Road Raipur D-mart   Mouthwash      3.0            6.0  
1       Ring Road Raipur D-mart    PCP / HC      0.0           16.0  
2       Ring Road Raipur D-mart  Toothbrush     20.0           26.0  
3       Ring Road Raipur D-mart  Toothpaste     42.0           59.0  
4            Seagull Appartment   Mouthwash     12.0           15.0

In [87]:
# summary_sosx = summary_sos[summary_sos['New_Amazing_EMP_ID'] == 'CAMP002']
# summary_sosx.head(20)

In [68]:
# sos_npf_master.info()